In [1]:
library("sf")
library("leaflet")

Warning message:
“il pacchetto ‘sf’ è stato creato con R versione 4.3.2”
Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE

Warning message:
“il pacchetto ‘leaflet’ è stato creato con R versione 4.3.2”


# Visualize Bucharest data

Quick visualization of the dataset for Bucharest.

In [2]:
bucharest_vector_path <- "../data/CRiSp-data-bucharest.gpkg"

In [3]:
st_layers(bucharest_vector_path)

Driver: GPKG 
Available layers:
              layer_name        geometry_type features fields
1          OSM_buildings     3D Multi Polygon    65300     38
2              OSM_roads 3D Multi Line String    39686     19
3        URC-D_buildings     3D Multi Polygon    11769     38
4 URC-D_dambovita_l_2017 3D Multi Line String        7     15
5 URC-D_dambovita_p_2017     3D Multi Polygon      226     27
6            URC-D_roads 3D Multi Line String     6595     23
7         URC-D_segments        Multi Polygon        9      4
8   URC-D_valleyedge_mod        Multi Polygon        4     10
9           layer_styles                   NA        8     12
                            crs_name
1 ETRS89 Lambert Azimutal Equal Area
2 ETRS89 Lambert Azimutal Equal Area
3           Undefined geographic SRS
4 ETRS89 Lambert Azimutal Equal Area
5 ETRS89 Lambert Azimutal Equal Area
6 ETRS89 Lambert Azimutal Equal Area
7 ETRS89 Lambert Azimutal Equal Area
8 ETRS89 Lambert Azimutal Equal Area
9              

In [4]:
loadLayer <- function(x) st_read(bucharest_vector_path, layer = x)

# dambovita lines - waterway
dambovita_l <-  loadLayer("URC-D_dambovita_l_2017") |>
    st_zm(drop = TRUE, what = "ZM")  # drop Z dimension

# dambovita polygons - areas labelled as water
dambovita_p <- loadLayer("URC-D_dambovita_p_2017") |>
    st_zm(drop = TRUE, what = "ZM")  # drop Z dimension

# roads
roads <- loadLayer("URC-D_roads") |>
    st_zm(drop = TRUE, what = "ZM")  # drop Z dimension

# corridor segments
segments <- loadLayer("URC-D_segments")

# valleyedge
valleyedge <- loadLayer("URC-D_valleyedge_mod")

# buildings
buildings <- loadLayer("URC-D_buildings") |>
    st_zm(drop = TRUE, what = "ZM") |>  # drop Z dimension
    st_set_crs(st_crs(segments))  # the CRS is not set

Reading layer `URC-D_dambovita_l_2017' from data source 
  `/Users/fnattino/Projects/CRiSp/Repos/prototyping/data/CRiSp-data-bucharest.gpkg' 
  using driver `GPKG'
Simple feature collection with 7 features and 15 fields
Geometry type: MULTILINESTRING
Dimension:     XYZ
Bounding box:  xmin: 5582298 ymin: 2504134 xmax: 5609111 ymax: 2508674
z_range:       zmin: 0 zmax: 0
Projected CRS: ETRS89 Lambert Azimutal Equal Area
Reading layer `URC-D_dambovita_p_2017' from data source 
  `/Users/fnattino/Projects/CRiSp/Repos/prototyping/data/CRiSp-data-bucharest.gpkg' 
  using driver `GPKG'
Simple feature collection with 226 features and 27 fields
Geometry type: MULTIPOLYGON
Dimension:     XYZ
Bounding box:  xmin: 5584029 ymin: 2502846 xmax: 5604608 ymax: 2508952
z_range:       zmin: 0 zmax: 0
Projected CRS: ETRS89 Lambert Azimutal Equal Area
Reading layer `URC-D_roads' from data source 
  `/Users/fnattino/Projects/CRiSp/Repos/prototyping/data/CRiSp-data-bucharest.gpkg' 
  using driver `GPKG'
Simp

Warning message:
“st_crs<- : replacing crs does not reproject data; use st_transform for that”


In [5]:
getGeomLatLong <- function(x) st_transform(x, 4326) |> st_geometry()

groups = c(
    "roads", 
    "buildings", 
    "water polygons", 
    "water linestrings", 
    "valley edge", 
    "segments"
)

leaflet() |>
    addTiles() |>
    addPolylines(data = getGeomLatLong(roads), color = "black", group = groups[1]) |>
    # addPolygons(data = getGeomLatLong(buildings), color = "magenta", group = groups[2]) |>
    addPolygons(data = getGeomLatLong(dambovita_p), color = "cyan", group = groups[3]) |>
    addPolylines(data = getGeomLatLong(dambovita_l), color = "blue", group = groups[4]) |>
    addPolygons(data = getGeomLatLong(valleyedge), color = "yellow", group = groups[5]) |>
    addPolygons(data = getGeomLatLong(segments), color = "red", fill = FALSE, group = groups[6]) |>
    addLayersControl(overlayGroups=groups)

HTML widgets cannot be represented in plain text (need html)